# IMDb Movie Reviews Sentiment Analysis using BERT
This notebook demonstrates how to fine-tune a pre-trained BERT model for sentiment analysis on the IMDb movie reviews dataset. We will use the Hugging Face Transformers library and datasets to achieve this.

In [ ]:
!pip install transformers datasets

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch

In [ ]:
# Load IMDb dataset
dataset = load_dataset("imdb")

In [ ]:
# Load the pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

In [ ]:
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
# Split the dataset into train and test sets
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))  # using a small subset for demonstration
test_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [ ]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
)

In [ ]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
# Train the model
trainer.train()

In [ ]:
# Evaluate the model
results = trainer.evaluate()
results